EP02.2 - Visão e Processamento de Imagens
===

Nº USP | Nome
---|---|
11347794|Aarão Melo Lopes|

---

## Acesso


Se estiver utilizando o Google Colaboratory execute os passos a seguir. No entanto, se estiver rodando em um dispositivo local, ignore esta seção.


Realize login com sua conta google, para ter acesso aos arquivos compartilhados, executando a célula abaixo.

In [67]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Mude o diretório corrente para a pasta do projeto.

In [ ]:
cd drive/Shared drives/EP02 - Visão e Processamento de Imagens

## Geração


### Normalização

In [69]:
def histogram(image):
  histogram = [0 for j in range(256)]
  for i in range(1280):
    for j in range(720):
      histogram[image[i][j][0]] += 1 
  return histogram

In [ ]:
import numpy as np
def equalization(image):
  equalization =  [[[0,0,0] for j in range(720)] for i in range(1280)]
  histog= histogram(image)
  for i in range(1280):
    for j in range(720):
      sum = 0
      for k in range(image[i][j][0]+1):
        sum += histog[k] 
      color = sum*255/(1280*720) 
      equalization[i][j] = [color,color,color]
  return np.uint8(equalization)
  

In [ ]:
from skimage.io import imread, imsave
for i in range(1080):
  im = imread("augmentedDataset/grayScale/" + str(i+1) + '.jpeg')
  equal = equalization(im)
  del im
  imsave('normalizedDataset/grayScale/'+str(i+1) + '.jpeg', equal)
  del equal
  print("save equalization gray " + str(i+1) )

In [ ]:
from skimage.io import imread, imsave
for i in range(1080):
  im = imread("augmentedDataset/logScale/" + str(i+1) + '.jpeg')
  equal = equalization(im)
  del im
  imsave('normalizedDataset/logScale/'+str(i+1) + '.jpeg', equal)
  del equal
  print("save equalization log " + str(i+1) )

In [ ]:
from skimage.io import imread, imsave
for i in range(1080):
  im = imread("augmentedDataset/meanScale/" + str(i+1) + '.jpeg')
  equal = equalization(im)
  del im
  imsave('normalizedDataset/meanScale/'+str(i+1) + '.jpeg', equal)
  del equal
  print("save equalization mean " + str(i+1) )

In [ ]:
from skimage.io import imread, imsave
for i in range(1080):
  im = imread("augmentedDataset/powScale/" + str(i+1) + '.jpeg')
  equal = equalization(im)
  del im
  imsave('normalizedDataset/powScale/'+str(i+1) + '.jpeg', equal)
  del equal
  print("save equalization pow " + str(i+1) )

In [ ]:
from skimage.io import imread, imsave
for i in range(1080):
  im = imread("augmentedDataset/shadowScale/" + str(i+1) + '.jpeg')
  equal = equalization(im)
  del im
  imsave('normalizedDataset/shadowScale/'+str(i+1) + '.jpeg', equal)
  del equal
  print("save equalization shadow " + str(i+1) )

In [ ]:
from skimage.io import imread, imshow, imsave
import matplotlib.pyplot as plt
e = equalization(grayScale[0])
h = histogram(e)
plt.bar(range(256),h)
plt.show()
imshow(e)

In [ ]:
from skimage.io import imread, imshow, imsave
import matplotlib.pyplot as plt
h = histogram(grayScale[0])
plt.bar(range(256),h)
plt.show()
imshow(grayScale[0])

### Protótipo Médio

In [3]:
def darkImage(): return [[[0, 0, 0] for j in range(720)] for i in range(1280)]

In [30]:
import numpy as np
def mean(images):
  size = len(images)
  result = darkImage()
  for i in range(1280):
    for j in range(720):
      sum = 0
      for k in range(size):
        sum += float(images[k][i][j][0])
      color = min(255, round(sum/size))
      result[i][j] = [color,color,color]
  return np.uint8(result) 

In [98]:
import csv
classes = ["copo", "livro", "caneta", "colher", "chinelo", "chave", "garfo", "isqueiro", "faca", "xicara"]
files = [[] for i in range(10)]

with open("Metadados.csv", "r") as arquivo_csv:
    leitor = csv.reader(arquivo_csv, delimiter=",")
    for coluna in leitor:
      classe = coluna[1]
      nome = coluna[0]
      for i in range(10):
        if classe == classes[i]:
          files[i].append(nome)

In [ ]:
from skimage.io import imread, imsave,imshow
for dataset in ["augmentedDataset/","normalizedDataset/"]:
  for scale in ["grayScale/", "logScale/", "meanScale/", "powScale/", "shadowScale/"]:
    i=0
    for classe in classes:
      images=[]
      for file in files[i]:
        images.append(imread( dataset + scale +file ))
      im = mean(images) 
      imsave("meanPrototypes/" + dataset + scale + classe + ".jpeg", im)
      print("save " + "meanPrototypes/" + dataset + scale + classe + ".jpeg")
      i+=1

### Histograma médio 

In [ ]:
from skimage.io import imread, imsave,imshow
import numpy

for dataset in ["augmentedDataset/","normalizedDataset/"]:
  for scale in ["grayScale/", "logScale/", "meanScale/", "powScale/", "shadowScale/"]:
    i=0
    for classe in classes:
      meanHistogram=[0 for k in range(256)]
      for file in files[i]:
        h = histogram(imread( dataset + scale + file ))
        for j in range(256):
          meanHistogram[j] += h[j]
      for j in range(256):
        meanHistogram[j] = round(meanHistogram[j]/108)
      numpy.savetxt("meanHistograms/" + dataset + scale + classe + ".csv", numpy.asarray([meanHistogram]), delimiter=",", fmt='%d')
      print("save " + "meanHistograms/" + dataset + scale + classe + ".csv")
      i+=1

### Variância

In [ ]:
from skimage.io import imread, imsave,imshow
import numpy
import math
for dataset in ["augmentedDataset/","normalizedDataset/"]:
  for scale in ["grayScale/", "logScale/", "meanScale/", "powScale/", "shadowScale/"]:

    i=0
    for classe in classes:
      varianceHistogram=[0 for k in range(256)]
      mean = [ 0 for j in range(256)]

      with open("meanHistograms/" + dataset + scale + classe + ".csv", "r") as arquivo_csv:
        leitor = csv.reader(arquivo_csv, delimiter=",")
        for coluna in leitor:
          for j in range(256):
            mean[j] = int(coluna[j])

      for file in files[i]:
        h = histogram(imread( dataset + scale + file ))
        for j in range(256):
          varianceHistogram[j] += (h[j]-mean[j])**2
      for j in range(256):
        varianceHistogram[j] = round(math.sqrt(varianceHistogram[j]/108))

      numpy.savetxt("varianceHistograms/" + dataset + scale + classe + ".csv", numpy.asarray([varianceHistogram]), delimiter=",", fmt='%d')
      print("save " + "varianceHistograms/" + dataset + scale + classe + ".csv")
      i+=1

## Visualização


Visualização dos dados normalizados

In [ ]:
import matplotlib.pyplot as plt
from skimage.io import imread, imshow, imsave

def loadDir(path):
  dataset = []
  for i in range(1080):
    dataset.append(imread(path+"/" + str(i+1) + '.jpeg'))
    print('read ' + str(i+1) + '.jpeg')
  return dataset

def showimages(image_list):
  for j in range(40):
    fig, axes = plt.subplots(1, 27, figsize=(20, 20))
    ax = axes.ravel()
    for i in range(27):
        ax[i].imshow(image_list[i+j*27])
        ax[i].axis('off')
    fig.tight_layout()
    plt.show()

In [ ]:
grayScale = loadDir("normalizedDataset/grayScale")

In [ ]:
showimages(grayScale)

In [ ]:
powScale = loadDir("normalizedDataset/powScale")

In [ ]:
showimages(powScale)

In [ ]:
shadowScale = loadDir("normalizedDataset/shadowScale")

In [ ]:
showimages(shadowScale)

In [ ]:
logScale = loadDir("normalizedDataset/logScale")

In [ ]:
showimages(logScale)

In [ ]:
meanScale = loadDir("normalizedDataset/meanScale")

In [ ]:
showimages(meanScale)

Visualização dos protótipo médios


In [ ]:
from skimage.io import imread, imshow, imsave
def loadProto():
  data = []
  for dataset in ["augmentedDataset/","normalizedDataset/"]:
    for scale in ["grayScale/", "logScale/", "meanScale/", "powScale/", "shadowScale/"]:
      for classe in classes:
        data.append(imread("meanPrototypes/" + dataset + scale + classe + ".jpeg"))
  return data

In [ ]:
import matplotlib.pyplot as plt
classes = ["copo", "livro", "caneta", "colher", "chinelo", "chave", "garfo", "isqueiro", "faca", "xicara"]
def showProto(image_list):
  for j in range(10):
    fig, axes = plt.subplots(1, 10, figsize=(20, 20))
    ax = axes.ravel()
    for i in range(10):
        ax[i].imshow(image_list[i+j*10])
        ax[i].axis('off')
    fig.tight_layout()
    plt.show()

In [ ]:
showProto(loadProto())

Visualização dos histogramas médios 

In [ ]:
import numpy
import csv
meanHistograms = []
for dataset in ["augmentedDataset/","normalizedDataset/"]:
  for scale in ["grayScale/", "logScale/", "meanScale/", "powScale/", "shadowScale/"]:
    for classe in classes:
      mean = [ 0 for j in range(256)]
      with open("meanHistograms/" + dataset + scale + classe + ".csv", "r") as arquivo_csv:
        leitor = csv.reader(arquivo_csv, delimiter=",")
        for coluna in leitor:
          for j in range(256):
            mean[j] = int(coluna[j])
      meanHistograms.append(mean)
      

In [105]:
import matplotlib.pyplot as plt
def showHistograms(histogram_list):
  for j in range(10):
    fig, axes = plt.subplots(1, 10, figsize=(20, 5))
    ax = axes.ravel()
    for i in range(10):
        ax[i].bar(range(256),histogram_list[i+j*10])
        ax[i].axis('off')
    fig.tight_layout()
    plt.show()

In [ ]:
showHistograms(meanHistograms)

Visualização da variância

In [106]:
import numpy
import csv
varianceHistograms = []
for dataset in ["augmentedDataset/","normalizedDataset/"]:
  for scale in ["grayScale/", "logScale/", "meanScale/", "powScale/", "shadowScale/"]:
    for classe in classes:
      var = [ 0 for j in range(256)]
      with open("varianceHistograms/" + dataset + scale + classe + ".csv", "r") as arquivo_csv:
        leitor = csv.reader(arquivo_csv, delimiter=",")
        for coluna in leitor:
          for j in range(256):
            var[j] = int(coluna[j])
      varianceHistograms.append(var)

In [ ]:
showHistograms(varianceHistograms)